In [1]:
# imports

import os
import json
import ollama
from google import genai
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
    
OPENAI_MODEL = 'gpt-4o-mini'
GEMINI_MODEL = 'gemini-2.5-flash' 
OLLAMA_MODEL = 'llama3.2'

openai = OpenAI()
gemini = genai.Client(api_key = gemini_api_key)

tools = []
gemini_tools = []

cached_search = {
    ('delhi', 'delhi'): "INR 0",
}

convertion_rate_to_inr = {
    "USD": 85.81,
    "EUR": 100.25,
    "GBP": 115.90,
    "AUD": 56.43,
    "CAD": 62.70,
    "SGD": 67.05,
    "CHF": 107.79,
    "JPY": 0.5825,
    "CNY": 11.97,
    "AED": 23.37,
    "NZD": 51.56,
    "SAR": 22.88,
    "QAR": 23.58,
    "OMR": 222.89,
    "BHD": 227.62,
    "KWD": 280.90,
    "MYR": 20.18,
    "THB": 2.655,
    "HKD": 10.93,
    "ZAR": 4.79
}

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [3]:
import requests
from bs4 import BeautifulSoup


headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url 
        try:
            response = requests.get(url=self.url, headers=headers, timeout=10)
            response.raise_for_status()
            self.body = response.content    
        except requests.RequestException as e:
            print(f"Failed to fetch {self.url}: {e}")
            self.body = b""
            self.title = "Failed to load"
            self.text = ""
            self.links = []
            return
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(['script', 'style', 'img', 'input']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = "" 
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_content(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        


In [4]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

class GoogleSearch:
    def __init__(self, api_key=None, cse_id=None):
        """
        Initialize the Google Search Tool
        
        Args:
            api_key: Your Google API key (or set GOOGLE_API_KEY env var)
            cse_id: Your Custom Search Engine ID (or set GOOGLE_CSE_ID env var)
        """
        self.api_key = api_key or os.getenv('GOOGLE_SEARCH_KEY')
        self.cse_id = cse_id or os.getenv('GOOGLE_CSE_ID')

        if not self.api_key:
            raise ValueError("API key is required. Set GOOGLE_API_KEY env var or pass api_key parameter")
        if not self.cse_id:
            raise ValueError("CSE ID is required. Set GOOGLE_CSE_ID env var or pass cse_id parameter")
            
        self.service = build("customsearch", "v1", developerKey=self.api_key)
    
    def search(self, query: str, num_result: int=10, start_index: int=1):
        """
        Perform a Google Custom Search
        
        Args:
            query: Search query string
            num_results: Number of results to return (1-10)
            start_index: Starting index for results (for pagination)
            
        Returns:
            dict: Search results or None if error
        """
        try:
            res = self.service.cse().list(
                q=query,
                cx=self.cse_id,
                num=min(num_result, 10),
                start=start_index
            ).execute()

            return self._parse_results(res)
        except HttpError as e:
            print(f"HTTP Error: {e}")
            return None
        except Exception as e:
            print(f"Unexpected error: {e}")
            return None
        
    def _parse_results(self, raw_res):
        """Parse raw API response into clean format"""
        if "items" not in raw_res:
            return {
                'total_results': 0,
                'results': [],
                'search_info': raw_res.get('searchInformation', {})
            }
        
        parsed_items = []
        for item in raw_res["items"]:
            parsed_item = {
                "title": item.get("title", ''),
                "link": item.get("link", ''),
                "snippet": item.get("snippet", ''),
                "display_link": item.get("display_link", ''),
                'formatted_url': item.get('formattedUrl', '')
            }

            parsed_items.append(parsed_item)
        
        return {
            'total_results': int(raw_res.get('searchInformation', {}).get('totalResults', '0')),
            'results': parsed_items,
            'search_info': raw_res.get('searchInformation', {})
        }
    
    def compile_search_pages(self, query: str, num_result: int = 10, start_index: int=1):
        """
        Compiles a list of results from multiple search pages for a given query

        Args:
            query: Search query string
            num_results: Number of results to return (1-10)
            start_index: Starting index for results (for pagination)
            
        Returns:
            str: Concatenated results from all search pages for the given query
        """

        result = ""

        search_res = self.search(query=query, num_result=num_result, start_index=start_index)

        print(search_res)

        for item in search_res['results']:
            print(item.get('title'))
            result += f"\n\nTitle: {item.get('title', '')}\n"
            result += Website(item.get('link', '')).get_content()

        print(result)

        return result

google_search = GoogleSearch()

In [5]:
# google_search.compile_search_pages('flight ticket price from delhi to chandigarh', num_result=4)

In [6]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "Always ask the user about the departure point in case it asks about the price and departure is not mentioned."

In [7]:
def analyze_result_for_price(result: str, source: str, model: str):
    print("Analyze web results: ", source, model)

    system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a search query."
    system_prompt = "Provide the lowest price, highest price and average price for one way and round trips."
    system_prompt += "Always return the price in INR. If you are not sure about the conversion rate, only then use the following conversion rates:"
    system_prompt += f"{convertion_rate_to_inr} for conversion rates. Interpret the given conversion rate as for example:"
    system_prompt += "1 USD to INR = 85.81. Return result in Markdown"
    
    if source == 'ollama':
        model_to_use = model if model else OLLAMA_MODEL

        print(f"Using model: {model_to_use}\n\n")

        try:
            response = ollama.chat(
                model=model_to_use, 
                messages=[
                    {"role":"system", "content": system_prompt},
                    {"role": "user", "content": result}
                ],
            )
            
            result = response['message']['content']
            return result
        except Exception as e:
            print(f"An error occurred during the API call: {e}")
            return None
    elif source == 'openai':
        try:
            response = openai.chat.completions.create(
                model=OPENAI_MODEL,
                messages=[
                    {"role":"system", "content": system_prompt},
                    {"role":"user", "content": result}
                ],
                
            )

            result = response.choices[0].message.content
            return result
        except Exception as e:
            print(f"An error occurred during the API call: {e}")
            return None
    elif source == 'gemini':
        try:
            response = gemini.models.generate_content(
                model=GEMINI_MODEL,
                contents=f"{system_prompt}\n\n{result}"
            )

            result = response.text
            return result
        except Exception as e:
            print(f"An error occurred during the API call: {e}")
            return None
    else:
        print("Source not supported")

In [8]:
def get_ticket_price(destination_city, departure_city, source="openai", model=""):
    if not destination_city or not departure_city:
        return "Error: Both destination and departure cities are required"
    
    print(f"Tool get_ticket_price called for {destination_city} from {departure_city}")
    print("get_ticket_price: ", model)

    dest = destination_city.lower()
    dept = departure_city.lower()

    cache_key = (dest, dept)

    if cache_key not in cached_search:
        try:
            query = f'flight ticket price from {dept} to {dest}' 
            results = google_search.compile_search_pages(query=query, num_result=10) 
            
            if results:  # Check if results is not empty
                    cached_search[cache_key] = results
            else:
                return "Error: No search results found"
        except Exception as e:
            print(f"Error during search: {e}")
            return f"Error: Unable to fetch flight prices - {str(e)}"
    else:
        results = cached_search[cache_key]

    try:
        return analyze_result_for_price(results, source, model)
    except Exception as e:
        print(f"Error analyzing results: {e}")
        return f"Error: Unable to analyze price data - {str(e)}"


In [9]:
# Markdown(get_ticket_price('New York', 'London', "gemini", ""))

In [10]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the current flight ticket price between two cities. Call this whenever you need to know flight prices, for example when a customer asks 'How much is a ticket from Delhi to Mumbai?', 'What's the flight cost to Chandigarh?', or 'Show me ticket prices for travel between these cities'. This function searches for real-time flight pricing information from multiple sources.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to (e.g., 'Mumbai', 'Delhi', 'Chandigarh')",
            },
            "departure_city": {
                "type": "string",
                "description": "The city that the customer wants to travel from (e.g., 'Delhi', 'Mumbai', 'Bangalore')",
            },
            "source": {
                "type": "string",
                "description": "The AI model source to use for price analysis (optional, defaults to 'openai')",
                "default": "openai"
            },
            "model": {
                "type": "string", 
                "description": "The specific AI model to use for analysis (optional, defaults to empty string)",
                "default": ""
            }
        },
        "required": ["destination_city", "departure_city"],
        "additionalProperties": False
    }
}

tools.append({"type": "function", "function": price_function})

In [11]:
gemini_tools = [
    {
        "function_declarations": [
            {
                "name": "get_ticket_price",
                "description": "Get the current flight ticket price between two cities. Call this whenever you need to know flight prices, for example when a customer asks 'How much is a ticket from Delhi to Mumbai?', 'What's the flight cost to Chandigarh?', or 'Show me ticket prices for travel between these cities'. This function searches for real-time flight pricing information from multiple sources.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "destination_city": {
                            "type": "string",
                            "description": "The city that the customer wants to travel to (e.g., 'Mumbai', 'Delhi', 'Chandigarh')"
                        },
                        "departure_city": {
                            "type": "string",
                            "description": "The city that the customer wants to travel from (e.g., 'Delhi', 'Mumbai', 'Bangalore')"
                        },
                        "source": {
                            "type": "string",
                            "description": "The AI model source to use for price analysis (optional, defaults to 'openai')"
                        },
                        "model": {
                            "type": "string",
                            "description": "The specific AI model to use for analysis (optional, defaults to empty string)"
                        }
                    },
                    "required": ["destination_city", "departure_city"]
                }
            }
        ]
    }
]

In [ ]:
def handle_tool_call(message, model):

    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    print(tool_call)
    if tool_call.function.name == "get_ticket_price":
        dest_city = arguments.get("destination_city", '')
        dept_city = arguments.get("departure_city",'')
        price = get_ticket_price(dest_city, dept_city, model, "")
        return {
            "role": "tool",
            "content": json.dumps({"destination_city": dest_city,"departure_city": dept_city,"price": price}),
            "tool_call_id": tool_call.id
        }
    return None

def handle_tool_call_gemini(response, model):
    tool_call = response.candidates[0].content.parts[0].function_call
    function_name = tool_call.name
    arguments = tool_call.args
    
    if function_name == "get_ticket_price":
        dest_city = arguments.get("destination_city", "")
        dept_city = arguments.get("departure_city", "")
        price = get_ticket_price(dest_city, dept_city, model, "")
        
        return {
            "tool_response": {
                "name": function_name,
                "response": {
                    "content": json.dumps({
                        "destination_city": dest_city,
                        "departure_city": dept_city,
                        "price": price
                    })
                }
            }
        }
    
    return None

In [ ]:
def chat(history, model):
    MODEL_TO_USE = ""
    if model.lower() == 'openai':
        MODEL_TO_USE = OPENAI_MODEL

        messages = [{"role": "system", "content": system_message}] + history
        response = openai.chat.completions.create(model=MODEL_TO_USE, messages=messages, tools=tools)

        if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            response = handle_tool_call(message, model.lower())
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL_TO_USE, messages=messages, tools=tools)
        
        history += [{"role": "assistant", "content": response.choices[0].message.content}]
    elif model.lower() == 'gemini':
        MODEL_TO_USE = GEMINI_MODEL
        messages = [{"role": "system", "content": system_message}] + history
        response = gemini.models.generate_content(messages, tools=gemini_tools) 
        candidate = response.candidates[0]
        
        if candidate.finish_reason == 'TOOL_CALL':
            messages.append(candidate.content)
            tool_response = handle_tool_call_gemini(response, model.lower())
            messages.append(tool_response)
            response = gemini.models.generate_content(messages, tools=gemini_tools)
        
        history += [{"role": "model", "content": response.text}]
    return history

In [16]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
        model = gr.Dropdown(["OpenAI", "Gemini", "Ollama"], label="Choose a model")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[chatbot, model], outputs=[chatbot]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessageToolCall(id='call_8hiqoroOg9ZdmJFHbXJKfkfx', function=Function(arguments='{"destination_city":"Kathmandu","departure_city":"Delhi"}', name='get_ticket_price'), type='function')
Tool get_ticket_price called for Kathmandu from Delhi
get_ticket_price:  
{'total_results': 6220000, 'results': [{'title': 'Find Cheap Flights from New Delhi to Kathmandu - Google Flights', 'link': 'https://www.google.com/travel/flights/flights-from-new-delhi-to-kathmandu.html', 'snippet': 'Popular airlines from New Delhi to Kathmandu · Air India. Nonstop. from $145. Typical price: $160–375 · IndiGo. Nonstop. from $148. Typical price: $150–365.', 'display_link': '', 'formatted_url': 'https://www.google.com/.../flights/flights-from-new-delhi-to-kathmandu.ht...'}, {'title': 'Book Delhi to Kathmandu Flights from INR 5,352 | Air India', 'link': 'https://www.airindia.com/en-us/book-flights/delhi-to-kathmandu-flights', 'snippet': 'Delhi to Kathmandu flight fare, Business class: INR 3,00,978 | Econ

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2229, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1740, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/

In [15]:
cached_search

{('delhi', 'delhi'): 'INR 0'}